In [2]:
import torch
import torch.nn as nn
from torch_geometric.data import Data
from typing import Dict

import ase
import ase.io
from ase import units
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import Stationary, ZeroRotation, MaxwellBoltzmannDistribution
from ase.calculators.calculator import Calculator, all_changes

import os
import time
import numpy as np
import pylab as pl


In [3]:
class DualReadoutMACE(nn.Module):
    """
    Finalna wersja klasy:
    1. Poprawnie rejestruje hak.
    2. Optymalizuje metodę forward.
    3. Inicjalizuje wagi głowy "delta" zerami dla poprawnego startu.
    """
    def __init__(self, base_mace_model: nn.Module):
        super().__init__()
        self.features = None
        self.mace_model = base_mace_model

        print("Zamrażanie parametrów całego modelu bazowego MACE...")
        for param in self.mace_model.parameters():
            param.requires_grad = False
        self.mace_model.eval()

        if not hasattr(self.mace_model, 'readouts') or not isinstance(self.mace_model.readouts, nn.ModuleList) or len(self.mace_model.readouts) == 0:
            raise AttributeError("Nie znaleziono modułu 'readouts' w modelu MACE lub jest on pusty.")

        self.base_readout_layer = self.mace_model.readouts[0]

        num_features = self.base_readout_layer.linear.irreps_in.dim
        print(f"Wykryto {num_features} cech wejściowych do głowy (readout).")

        self.delta_readout = nn.Linear(num_features, 1, bias=False)

        # INICJALIZACJA WAG ZERAMI
        torch.nn.init.zeros_(self.delta_readout.weight)
        print("Wagi nowej głowy 'delta_readout' zostały zainicjalizowane zerami.")

        self.base_readout_layer.register_forward_hook(self._hook_fn)
        print("Nowa głowa 'delta_readout' dodana. Hak poprawnie zarejestrowany.")

    def _hook_fn(self, module, input_data, output_data):
        self.features = input_data[0]

    def forward(self, data: Dict[str, torch.Tensor], compute_force: bool = False) -> Dict[str, torch.Tensor]:
        if compute_force:
            data["positions"].requires_grad_(True)

        base_output = self.mace_model(data, compute_force=False)
        base_energy = base_output["energy"]

        if self.features is None:
            raise RuntimeError("Hak nie przechwycił cech atomowych. Sprawdź strukturę modelu MACE.")

        delta_atomic_energies = self.delta_readout(self.features)
        delta_energy = torch.sum(delta_atomic_energies, dim=-2)

        self.features = None
        final_energy = base_energy + delta_energy

        output_data = {}
        if compute_force:
            forces = -torch.autograd.grad(
                outputs=final_energy.sum(),
                inputs=data["positions"],
                create_graph=False,
                retain_graph=False,
            )[0]
            output_data["forces"] = forces.detach()

        output_data["energy"] = final_energy.detach()
        return output_data

In [4]:
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- 1. Wczytaj modele ---
    MODEL_PATH = './MACE-MP_small.model'
    print(f"Wczytywanie modelu bazowego z: {MODEL_PATH}")
    base_model = torch.load(MODEL_PATH, map_location=device, weights_only=False).float()
    dual_model = DualReadoutMACE(base_model)

    # --- 2. Wydrukuj podsumowanie finalnego modelu ---
    print("\n" + "="*60)
    print("      Podsumowanie finalnego modelu 'DualReadoutMACE'")
    print("="*60)
    print(dual_model)
    print("-"*60)

    # --- 3. Wydrukuj szczegóły dotyczące parametrów trenowalnych vs zamrożonych ---
    total_params = sum(p.numel() for p in dual_model.parameters())
    trainable_params = sum(p.numel() for p in dual_model.parameters() if p.requires_grad)

    print(f"\nCałkowita liczba parametrów: {total_params:,}")
    print(f"Liczba trenowalnych parametrów: {trainable_params:,}")
    print("\nSzczegóły parametrów trenowalnych:")
    found_trainable = False
    for name, param in dual_model.named_parameters():
        if param.requires_grad:
            print(f"  - Warstwa: '{name}' | Rozmiar: {param.shape} | Status: Trenowalna")
            found_trainable = True
    
    if not found_trainable:
        print("  - Brak trenowalnych parametrów.")
        
    print("="*60)


Wczytywanie modelu bazowego z: ./MACE-MP_small.model


/home/krystynasyty/myenv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
Zamrażanie parametrów całego modelu bazowego MACE...
Wykryto 128 cech wejściowych do głowy (readout).
Wagi nowej głowy 'delta_readout' zostały zainicjalizowane zerami.
Nowa głowa 'delta_readout' dodana. Hak poprawnie zarejestrowany.

      Podsumowanie finalnego modelu 'DualReadoutMACE'
DualReadoutMACE(
  (mace_model): ScaleShiftMACE(
    (node_embedding): LinearNodeEmbeddingBlock(
      (linear): Linear(89x0e -> 128x0e | 11392 weights)
    )
    (radial_embedding): RadialEmbeddingBlock(
      (bessel_fn): BesselBasis(r_max=6.0, num_basis=10, trainable=False)
      (cutoff_fn): PolynomialCutoff(p=5.0, r_max=6.0)
    )
    (spherical_harmonics): SphericalHarmonics()
    (atomic_energies_fn): AtomicEnergiesBlock(energies=[[-3.6672, -1.3321, -3.4821, -4.7367, -7.7249, -8.4056, -7.3601, -7.2846, -4.8965, 0.0000, -2.7594, -2.8140, -4.8469, -7.6948, -6.9633, -4.6726, -2.8117, -0.0626, -2.61